In [1]:
# R to Python conversion of GLM simulation setup
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.preprocessing import StandardScaler

# Set parameters (matching R code exactly)
n = 500
p = 100
nonzero_coefs = 20
Amp = 9
rho = 0.6

# Generate Toeplitz correlation matrix (equivalent to toeplitz(rho^(0:(p-1))))
Theta_8 = np.array([[rho**abs(i-j) for j in range(p)] for i in range(p)])

# Generate X from multivariate normal (equivalent to mvrnorm)
X = multivariate_normal.rvs(mean=np.zeros(p), cov=Theta_8, size=n, random_state=42)

# Scale X with 1/sqrt(n) factor (equivalent to scale(X)/(sqrt(n)))
X = StandardScaler().fit_transform(X) / np.sqrt(n)

# Generate true coefficients (equivalent to beta setup)
beta = np.zeros(p)
beta[:nonzero_coefs] = np.random.choice([-Amp, Amp], size=nonzero_coefs, replace=True)

# Signal indices (equivalent to Signal_index <- 1:nonzero_coefs)
Signal_index = np.arange(nonzero_coefs)  # 0-indexed in Python vs 1-indexed in R

# True labels (equivalent to true_labels <- beta != 0)
true_labels = beta != 0

# Generate linear predictor (equivalent to linear_predictor <- X %*% beta)
linear_predictor = X @ beta

# Generate probabilities using logistic function (equivalent to prob <- 1 / (1 + exp(-linear_predictor)))
prob = 1 / (1 + np.exp(-linear_predictor))

# Generate binary response (equivalent to y <- rbinom(n, 1, prob))
y = np.random.binomial(1, prob, n)

print(f"Data shape: X={X.shape}, y={y.shape}")
print(f"True signal indices: {Signal_index}")
print(f"Number of true signals: {np.sum(true_labels)}")
print(f"y distribution: {np.sum(y)} ones out of {n} samples ({np.mean(y):.3f} proportion)")
print(f"Linear predictor range: [{np.min(linear_predictor):.3f}, {np.max(linear_predictor):.3f}]")
print(f"Probability range: [{np.min(prob):.3f}, {np.max(prob):.3f}]")

Data shape: X=(500, 100), y=(500,)
True signal indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Number of true signals: 20
y distribution: 246 ones out of 500 samples (0.492 proportion)
Linear predictor range: [-5.540, 4.820]
Probability range: [0.004, 0.992]


In [2]:
from nullstrap.models.glm import NullstrapGLM

# After the simulation code above...

# Initialize Nullstrap GLM
model = NullstrapGLM(
    fdr=0.1,           # Target FDR
    alpha_=None,       # Auto-select alpha (regularization parameter)
    B_reps=5,          # Bootstrap repetitions
    family="binomial", # GLM family
    random_state=42    # For reproducibility
)

# Fit the model
model.fit(X, y)

# Get results
threshold = model.threshold_
selected_features = model.selected_
statistics = model.statistic_
correction_factor = model.correction_factor_
alpha_used = model.alpha_used_

print(f"Threshold: {threshold}")
print(f"Selected features: {selected_features}")
print(f"Number selected: {len(selected_features)}")
print(f"Correction factor: {correction_factor}")
print(f"Alpha used: {alpha_used}")


Threshold: 3.752759457568664
Selected features: [ 0  1  2  3  4  6 11 12 17 18 96]
Number selected: 11
Correction factor: 1.377604161270841
Alpha used: 0.7187627327609252


In [3]:
from sklearn.linear_model import LogisticRegressionCV
import numpy as np

# Fit Logistic Regression with cross-validation to select lambda
logistic = LogisticRegressionCV(
    cv=5, 
    penalty='l1', 
    solver='liblinear',
    fit_intercept=False,
    random_state=42
).fit(X, y)

# Get the coefficients
logistic_coef = logistic.coef_.flatten()

# Select features with nonzero coefficients
logistic_selected = np.where(logistic_coef != 0)[0]

print(f"Logistic selected features: {logistic_selected}")
print(f"Number selected by Logistic: {len(logistic_selected)}")
print(f"Logistic C (1/lambda) used: {logistic.C_[0]}")
print(f"Logistic lambda used: {1.0 / logistic.C_[0]}")


Logistic selected features: [ 0  1  2  3  4  6  7  8 10 11 12 16 17 18 19 20 23 28 29 30 32 34 36 38
 41 42 44 46 58 61 66 72 75 79 85 87 91 96]
Number selected by Logistic: 38
Logistic C (1/lambda) used: 2.782559402207126
Logistic lambda used: 0.3593813663804626


In [4]:
from nullstrap.utils.metrics import compute_power, compute_fdr

# Compute true support
true_support = np.where(beta != 0)[0]

# Nullstrap GLM results
nullstrap_power = compute_power(selected_features, true_support)
nullstrap_fdr = compute_fdr(selected_features, true_support, p) 
print(f"Nullstrap GLM Power: {nullstrap_power:.3f}")
print(f"Nullstrap GLM FDR: {nullstrap_fdr:.3f}")

# Logistic Regression results
logistic_power = compute_power(logistic_selected, true_support)
logistic_fdr = compute_fdr(logistic_selected, true_support, p)
print(f"Logistic Power: {logistic_power:.3f}")
print(f"Logistic FDR: {logistic_fdr:.3f}")

# Additional GLM-specific metrics
print(f"\nClass distribution: {np.sum(y)}/{n} positive cases ({np.mean(y):.3f} proportion)")
print(f"True positive rate: {np.sum(selected_features < nonzero_coefs)}/{nonzero_coefs} true signals selected")
print(f"False positive rate: {np.sum(selected_features >= nonzero_coefs)}/{len(selected_features)} selected features are false positives")


Nullstrap GLM Power: 0.500
Nullstrap GLM FDR: 0.091
Logistic Power: 0.750
Logistic FDR: 0.605

Class distribution: 246/500 positive cases (0.492 proportion)
True positive rate: 10/20 true signals selected
False positive rate: 1/11 selected features are false positives
